In [ ]:
# question include new mode namley start_cv_run_building_id
# or introduces new parameter that can be use in the config file
# the latter is probably better
# pseudo code:
# start_cv_run_buidlingwise:
# if mode = group_cv_building
    # meter0 = train_df.meter == 0 returns logical vector
    # train_df = train_df meter0 rows
    # label = label aber nur meter 0 rows
# group kfold
# define the groups array by building_id
# and run the cross-validation

In [ ]:
# building_id keepen auch wenn dropped in make features
# if cv grouped on building id
# and in drop vector
# then delete it from drop vector

# summ we have a vector containing the building id
# this vector indicates which variables will be dropped
# but we nee building id to group the folds in the cross validation
# first solution see above
#
# also it needs to be dropped again in the cross validation function so taht it is
# not inculded in the parameeters for tehe mdoel

In [ ]:
# label = adden

In [3]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
import yaml

In [7]:
import numpy as np

In [ ]:
def main(mode, input_filepath, output_filepath):
    """
    Collects prepared data and starts training an LightGBM model. Parameters
    can be specified by editing src/config.yml.
    :param mode: Specifies mode to run. Options are full (no validation set,
    single fold), cv (cross validation), by_meter (training by meter type),
    by_building (training by building id).
    :param input_filepath: Directory that contains the processed data.
    :param output_filepath: Directory that will contain the trained models.
    """
    random.seed(1337)
    with timer("Loading processed training data"):
        train_df, label = load_processed_training_data(input_filepath)

    ###########################################################################
    # DEFINE PARAMETERS FOR THE LGBM MODEL                                     #
    ###########################################################################
    with open("src/config.yml", 'r') as ymlfile:
        cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    params = cfg["lgbm_params"]
    num_boost_round = cfg["lgbm_num_boost_round"]
    early_stopping_rounds = cfg["lgbm_early_stopping_rounds"]
    splits = cfg["lgbm_splits_for_cv"]
    verbose_eval = cfg["lgbm_verbose_eval"]
    grouped_on_building = cfg["lgbm_cv_grouped_on_building"]
    ###########################################################################

    if mode == "full":
        start_full_training_run(train_df, label, params, verbose_eval,
                                num_boost_round, early_stopping_rounds,
                                output_filepath)
    elif mode == "cv":
        start_cv_run(train_df, label, params, splits, verbose_eval,
                     num_boost_round, early_stopping_rounds, output_filepath, grouped_on_building)
    elif mode == "by_meter":
        start_full_by_meter_run(train_df, label, params, verbose_eval,
                                num_boost_round, early_stopping_rounds, output_filepath)
    elif mode == "by_building":
        start_full_by_building_run(train_df, label, params, splits, verbose_eval,
                                   num_boost_round, early_stopping_rounds, output_filepath)
    else:
        raise ValueError("Choose a valid mode: 'full', 'cv'")

In [ ]:
elif mode == "cv":
    start_cv_run(train_df, label, params, splits, verbose_eval,
    num_boost_round, early_stopping_rounds, output_filepath, grouped_on_building)

In [ ]:
def start_cv_run(train_df, label, params, splits, verbose_eval,
                 num_boost_round, early_stopping_rounds, output_filepath,
                 grouped_on_building):
    """
    Starts a Cross Validation Run with the parameters provided.
    Scores will be documented and models will be saved.
    :param train_df: DataFrame which contains the training data.
    :param label: A vector which contains the labels of the training data.
    :param params: Dictionary with the model parameters
    :param splits: Integer describing the number of folds / splitting fraction.
    :param verbose_eval: The interval where training information is printed
    to console.
    :param num_boost_round: Maximum number of rounds / estimators for the training.
    :param early_stopping_rounds: If no improvement of the validation score in
    n rounds occur, the training will be stopped.
    :param output_filepath: Directory that will contain the trained models.
    """
    if grouped_on_building:
        if not 'building_id' in train_df.columns:
            raise ValueError(
                "For grouped cv, the cross validation is grouped on building_id."
                "Therefore it must be excluded from the drop section in the config file."
                "Note that the building_id is still not included in the model,"
                "it is needed for specifying the folds only and will be dropped afterwards.")
        output_filepath = output_filepath + "grouped_cv"
        is_meter0 = (train_df.meter == 0).values
        train_df = train_df.iloc[is_meter0,]
        label = label.iloc[is_meter0,]
        groups = train_df.building_id
        train_df = drop_columns(train_df, 'building_id')
        gkf = GroupKFold(n_splits = splits)
        indices = gkf.split(train_df, label, groups)
    else:
        output_filepath = output_filepath + "_cv"
        kf = Kfold(n_splits = splits, shuffle = False, random = state = 1337)
        indices = kf.split(train_df, label)
    cv_results = []
    with timer("Performing " + str(splits) + " fold cross-validation"):
        for i, (train_index, test_index) in enumerate(indices):
            with timer("~~~~ Fold %d of %d ~~~~" % (i + 1, splits)):
                x_train, x_valid = train_df.iloc[train_index], train_df.iloc[test_index]
                y_train, y_valid = label[train_index], label[test_index]

                train_lgb_df = lgb.Dataset(data=x_train, label=y_train)
                valid_lgb_df = lgb.Dataset(data=x_valid, label=y_valid)

                valid_sets = [train_lgb_df, valid_lgb_df]
                evals_result = dict()
                lgbm_model = lgb.train(params=params,
                                       train_set=train_lgb_df,
                                       num_boost_round=num_boost_round,
                                       valid_sets=valid_sets,
                                       valid_names=["train_loss", "eval"],
                                       verbose_eval=verbose_eval,
                                       evals_result=evals_result,
                                       early_stopping_rounds=early_stopping_rounds)
                save_model(output_filepath, lgbm_model)

                cv_results.append(evals_result)
        evaluate_cv_results(cv_results)


In [5]:
help(KFold)

Help on class KFold in module sklearn.model_selection._split:

class KFold(_BaseKFold)
 |  KFold(n_splits='warn', shuffle=False, random_state=None)
 |  
 |  K-Folds cross-validator
 |  
 |  Provides train/test indices to split data in train/test sets. Split
 |  dataset into k consecutive folds (without shuffling by default).
 |  
 |  Each fold is then used once as a validation while the k - 1 remaining
 |  folds form the training set.
 |  
 |  Read more in the :ref:`User Guide <cross_validation>`.
 |  
 |  Parameters
 |  ----------
 |  n_splits : int, default=3
 |      Number of folds. Must be at least 2.
 |  
 |      .. versionchanged:: 0.20
 |          ``n_splits`` default value will change from 3 to 5 in v0.22.
 |  
 |  shuffle : boolean, optional
 |      Whether to shuffle the data before splitting into batches.
 |  
 |  random_state : int, RandomState instance or None, optional, default=None
 |      If int, random_state is the seed used by the random number generator;
 |      If R

In [8]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
groups = np.array([0, 0, 2, 2])
group_kfold = GroupKFold(n_splits=2)
group_kfold.get_n_splits(X, y, groups)

2

In [9]:
print(group_kfold)

GroupKFold(n_splits=2)


In [9]:
for train_index, test_index in group_kfold.split(X, y, groups):
    print("TRAIN:", train_index, "TEST", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [0 1] TEST [2 3]
TRAIN: [2 3] TEST [0 1]


In [17]:
 for a, (b, c) in enumerate(group_kfold.split(X, y, groups)):
        print([a, (b,c)])

[0, (array([0, 1]), array([2, 3]))]
[1, (array([2, 3]), array([0, 1]))]


In [12]:
with open("../src/config.yml", 'r') as ymlfile:
        cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

In [13]:
cfg["drop"]

['timestamp',
 'month',
 'sea_level_pressure',
 'wind_direction',
 'wind_speed',
 'timezone',
 'country_code',
 'location',
 'site_id',
 'building_id']